![nvidia](images/nvidia.png)

# Combining Chains

In this notebook you'll learn how to compose multiple LLM-related chains.

---

## Objectives

By the time you complete this notebook you will:

- Learn how to compose chains of chains
- Apply your ability to chain meaningful language tasks.

---

## Imports

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnableParallel

---

## Create a Model Instance

In [ ]:
base_url = 'http://llama:8000/v1'
model = 'meta/llama-3.1-8b-instruct'
llm = ChatNVIDIA(base_url=base_url, model=model, temperature=0)

---

## Combining Multiple LLM Chains

If you recall, runnables can be composed into chains, but also, chains are themselves runnables. Therefore, chains can be used to compose larger chains.

It's easy to imagine tasks we would like to perform that would require multiple calls to an LLM for the desired end result. We'll begin our exploration of chaining chains with such a scenario, where we will compose multiple LLM chains, piping the output of one chain into the next.

To do this we are going to work with the following list of `thesis_statements`. Note: any typos you see in the thesis statements are intentional.

In [ ]:
thesis_statements = [
    "The fundametal concepts quantum physcis are difficult to graps, even for the mostly advanced students.",
    "Einstein's theroy of relativity revolutionised undrstanding of space and time, making it clear that they are interconnected.",
    "The first law of thermodynmics states that energy cannot be created or destoryed, excepting only transformed from one form to another.",
    "Electromagnetism is one of they four funadmental forces of nature, and it describes the interaction between charged particles.",
    "In the study of mechanic, Newton's laws of motion provide a comprehensive framework for understading the movement of objects under various forces."
]

Our goal is going to be to expand each of these thesis statements into a well-written paragraph, with the thesis statement itself being the first paragraph. You may have noticed, however, that each of these thesis statements contains spelling and/or grammar errors that need correcting.

Therefore, we are going to create a chain first to address the spelling and grammar issues, and then chain the corrected thesis statements into a second LLM chain responsible for generating the full paragraphs.

---

## Exercise: Create a Spelling and Grammar Chain

To begin, create `grammar_chain` which returns its inputs after performing spelling and grammar corrections on them.

We already have an LLM instance defined above (`llm`), but you will need to create both a prompt template and output parser to include in your chain.

You may need to develop your prompt template iteratively. Make sure especially that the chain returns only the corrected text, and not any additional comments etc. from the model.

Test your chain by sending it the batch of `thesis_statements` defined above.

Check out the solution below if you get stuck.

### Your Work Here

In [ ]:
grammar_chain = 'TODO' # TODO: grammar_chain should return its inputs after performing spelling and grammar on them.

### Solution

We begin by engineering a prompt for spelling and grammar correction. We take care to be specific in our prompt that the model should generate only the corrected text with no addional comment or preface.

In [ ]:
spelling_and_grammar_template = ChatPromptTemplate.from_template("""Fix any spelling or grammatical issues in the following text. Return \
back the correct text and only the corrected text with no additional comment or preface. Text: {text}""")

Next we create an instance of a string output parser.

In [ ]:
parser = StrOutputParser()

All that's left to do is compose the chain...

In [ ]:
grammar_chain = spelling_and_grammar_template | llm | parser

...and pass the thesis statements to it in batch.

In [ ]:
corrected_texts = grammar_chain.batch(thesis_statements)

Looking at the corrected outputs, it appears that the model did an excellent job.

In [ ]:
for corrected_text in corrected_texts:
    print(corrected_text)

---

## Exercise: Create a Paragraph Generator Chain

Create a second chain called `paragraph_generator_chain`. Given a sentence as input, it should use that sentence as the first sentence of a paragraph which it should generate.

**Note:** this chain should not contain any grammar or spell checking functionality. The chain should be responsible only for the paragraph generation task.

Test your chain by sending it the batch of `thesis_statements` defined above.

Feel free to check out the *Solution* below if you get stuck.

### Your Work Here

In [ ]:
paragraph_generator_chain = 'TODO'

### Solution

We begin the task by engineering a prompt.

In [ ]:
paragraph_generator_template = ChatPromptTemplate.from_template("""Generate a 4 to 8 sentence paragraph that begins with the following \
thesis statement. Return back the paragraph and only the paragrah with no addional comment or preface. Thesis statement: {thesis}""")

Since we already have a model instance and parser, all we have to do is compose the chain...

In [ ]:
paragraph_generator_chain = paragraph_generator_template | llm | parser

...and send it the batch of thesis statements.

In [ ]:
paragraphs = paragraph_generator_chain.batch(thesis_statements)

Looking at the generated paragraphs, it looks like the model did a great job.

It's worth highlighing that even though we did not prompt the model to address spelling and grammar mistakes, it did fix some of the spelling mistakes anyway, however, it's clear that most of the grammar errors from the thesis statements we passed in are still present.

In [ ]:
for paragraph in paragraphs:
    print(paragraph+'\n')

---

## Exercise: Create a Chain of Chains

Reusing the chains you've already created, create a `corrected_generator_chain` that uses the LLM first to perform spelling and grammar corrections on `thesis_statements` before then generating full paragraphs based the (corrected) thesis statements.

You don't need to overthink this. Just remember, chains are runnables, and can be piped together just like any other runnable.

Test your chain by sending it the batch of `thesis_statements` defined above.

Feel free to check out the *Solution* below if you get stuck.

### Your Work Here

### Solution

All we have to do to create our larger chain is to pipe together the 2 chains we already created.

In [ ]:
corrected_generator_chain = grammar_chain | paragraph_generator_chain

Just because it will be interesting, we can take a look at the computational graph for our new chain.

In [ ]:
print(corrected_generator_chain.get_graph().draw_ascii())

We can batch send our thesis statements to this larger chain just as we did with the smaller chains.

In [ ]:
paragraphs = corrected_generator_chain.batch(thesis_statements)

Looking at the final outputs we can see that the paragraphs were well-generated, but also, that all the spelling and grammar mistakes in the original thesis statements have been addressed.

In [ ]:
for paragraph in paragraphs:
    print(paragraph+'\n')

---

## Summary

In this notebook you learned to treat chains as the runnable they are and combine them together, including in ways that allowed you to leverage LLMs multiple times to accomplish a desired task.

In the next notebook you'll continue on the theme of chain composition, but this time focusing on the ability to create and utilize parallel chains.